### Dependencies

In [1]:
!pip install datasets
!pip install numpy soundfile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
from huggingface_hub import login

from datasets import load_dataset, Dataset, DatasetDict
from transformers import Wav2Vec2Processor, Wav2Vec2ForSequenceClassification, TrainingArguments, Trainer
import pandas as pd
import numpy as np
import torchaudio

### Load dataset

In [3]:
# Te pedirá ingresar tu token de autenticación de Hugging Face
login()

In [4]:
cv_1_train = load_dataset("mozilla-foundation/common_voice_6_1", "en", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for mozilla-foundation/common_voice_6_1 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_6_1.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating other split: 0 examples [00:00, ? examples/s]

Generating invalidated split: 0 examples [00:00, ? examples/s]

In [42]:
cv_1_test = load_dataset("mozilla-foundation/common_voice_6_1", "en", split="test")
cv_1_validation = load_dataset("mozilla-foundation/common_voice_6_1", "en", split="validation")

### Preprocess dataset

In [64]:
train_df = cv_1_train.to_pandas()
test_df = cv_1_test.to_pandas()
validation_df = cv_1_validation.to_pandas()

In [65]:
### Equilibrar por Acento y Genero

# Concatenar los DataFrames de train, validation y test
# df = pd.concat([train_df, validation_df, test_df], ignore_index=True)

df = train_df.sample(n=60000, random_state=3)

# Filtrar por los valores de género (Male y Female) y por acento (US y England)
df = df[df['gender'].isin(['male', 'female']) & df['accent'].isin(['us', 'england'])]

# Contar el número de muestras por combinación de género y acento
group_counts = df.groupby(['gender', 'accent']).size().reset_index(name='count')

# Encontrar el número mínimo de muestras en los subgrupos
min_count = group_counts['count'].min()

# Filtrar y equilibrar el DataFrame por cada combinación de género y acento
balanced_dfs = []
for (gender, accent), group in df.groupby(['gender', 'accent']):
    balanced_group = group.sample(n=min_count, random_state=42)
    balanced_dfs.append(balanced_group)

# Combinar los DataFrames equilibrados
balanced_df = pd.concat(balanced_dfs, ignore_index=True)

# Verificar el resultado
balanced_df[['accent', 'gender']].value_counts()


accent   gender
england  female    754
         male      754
us       female    754
         male      754
Name: count, dtype: int64

Audio duration

In [ ]:
import torchaudio

# Función para obtener la duración del audio
def get_audio_duration(row):
    waveform, sample_rate = torchaudio.load(row['path'])
    duration_seconds = waveform.size(1) / sample_rate  # Calcular la duración en segundos
    return duration_seconds

# Aplicar la función a cada fila del DataFrame balanceado
balanced_df['duration'] = balanced_df.apply(get_audio_duration, axis=1)

# Calcular la duración promedio de los audios
average_duration = balanced_df['duration'].mean()

# Mostrar la duración promedio
print(f"La duración promedio de los audios es de {average_duration:.2f} segundos.")


La duración promedio de los audios es de 5.55 segundos.


In [66]:
### Equilibrar solamente por acento
# df = cv_1_train.to_pandas()

# # Filtrar acentos US y England
# us_df = df[df['accent'] == 'us']
# england_df = df[df['accent'] == 'england']

# # Equilibrar los datasets (mismo número de muestras)
# england_count = len(england_df)
# us_df = us_df.sample(n=england_count, random_state=42)

# # Combinar los datasets equilibrados
# balanced_df = pd.concat([us_df, england_df])


# ## Usar menos muestras de las que tenemos
# balanced_df = balanced_df.sample(n=3000)

# Cargar el procesador de Wav2Vec2 (solo una vez)
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")

# Modificar la función de procesamiento de audio
def load_audio(row):
    # Cargar el archivo de audio
    waveform, sample_rate = torchaudio.load(row['path'])

    # Resamplear si es necesario
    if sample_rate != processor.feature_extractor.sampling_rate:
        resampler = torchaudio.transforms.Resample(sample_rate, processor.feature_extractor.sampling_rate)
        waveform = resampler(waveform)

    # Definir la longitud máxima (por ejemplo, 32000 muestras, ajusta según la duración de tu audio)
    max_length = int(processor.feature_extractor.sampling_rate * 5)  # Ejemplo: 2 segundos de audio

    # Aplicar el preprocesamiento de Wav2Vec2 con padding y truncation
    inputs = processor(waveform.squeeze().numpy(),
                       sampling_rate=processor.feature_extractor.sampling_rate,
                       return_tensors="pt",
                       padding="max_length",  # Padding para un tamaño uniforme
                       truncation=True,       # Truncar si la secuencia es demasiado larga
                       max_length=max_length)  # Longitud máxima

    # Retornar los valores de entrada procesados
    return inputs.input_values[0].tolist()

# Aplicar la función de carga y procesamiento del audio
balanced_df['input_values'] = balanced_df.apply(load_audio, axis=1)


# Convertir etiquetas de acento en números
label2id = {'us': 0, 'england': 1}
id2label = {v: k for k, v in label2id.items()}
balanced_df['label'] = balanced_df['accent'].map(label2id)

# Convertir DataFrame a Dataset de Hugging Face
balanced_dataset = Dataset.from_pandas(balanced_df)

# Seleccionar solo las columnas necesarias para el modelo
balanced_dataset = balanced_dataset.remove_columns(['audio', 'path', 'up_votes', 'down_votes', 'age', 'gender', 'locale', 'segment', 'client_id', 'sentence', 'accent'])
if '__index_level_0__' in balanced_dataset.column_names:
    balanced_dataset = balanced_dataset.remove_columns(['__index_level_0__'])


# Dividir el dataset en entrenamiento, prueba y validación
train_testvalid = balanced_dataset.train_test_split(test_size=0.2)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']
})

# Verificar la estructura del dataset final
dataset


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['input_values', 'label'],
        num_rows: 2412
    })
    test: Dataset({
        features: ['input_values', 'label'],
        num_rows: 302
    })
    valid: Dataset({
        features: ['input_values', 'label'],
        num_rows: 302
    })
})

### Finetuning Wav2Vec

In [69]:
# Cargar el modelo preentrenado para clasificación de secuencias
model = Wav2Vec2ForSequenceClassification.from_pretrained(
    "facebook/wav2vec2-base",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id,
)

# Definir argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./wav2vec2-accent-classification",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    learning_rate=0.00001,
    weight_decay=0.00001,
    save_total_limit=2,
    remove_unused_columns=False,
    report_to="none",  # No reporting to Weights & Biases, etc.
)

# Definir las funciones de evaluación
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = np.sum(preds == labels) / len(labels)
    return {"accuracy": accuracy}

# Inicializar el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["valid"],
    tokenizer=processor,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated an

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.332929,0.917219
2,0.483600,0.306578,0.900662


KeyboardInterrupt: 

.+ 5000 data: 0.92 accuracy, 0.18 validation and training loss

### Testing


Evaluation

In [ ]:
results = trainer.evaluate(dataset["test"])
print(f"Accuracy: {results['eval_accuracy']:.4f}")


### Save model

In [ ]:
model.save_pretrained("./model-wav2vec2-accent-classificationV4")
processor.save_pretrained("./model-wav2vec2-accent-classificationV4")


### Load and make inference

In [ ]:
# Mezclar (barajar) las filas del DataFrame
shuffled_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Ver las primeras filas del DataFrame mezclado
shuffled_df.head()


,client_id,path,audio,sentence,up_votes,down_votes,age,gender,accent,locale,segment
0,344bf332f298134d3880f9e38710e4c7e7037157bd63a3...,/root/.cache/huggingface/datasets/downloads/ex...,"{'bytes': None, 'path': '/root/.cache/huggingf...",She also plays the guitar and lute.,2,0,twenties,female,england,en,
1,27684391c8f6095629c2c12a88cb47055fa3abc33d53e3...,/root/.cache/huggingface/datasets/downloads/ex...,"{'bytes': None, 'path': '/root/.cache/huggingf...","""He's totally daft about birds.""",2,0,teens,female,us,en,
2,500d18a3257795d8ea82b3c39440f7656ecda7b29519f3...,/root/.cache/huggingface/datasets/downloads/ex...,"{'bytes': None, 'path': '/root/.cache/huggingf...",The proceeds of the book benefited several cha...,3,0,thirties,female,us,en,
3,2429806e5f5039a19ed9376741f82f6512e720f71bd07f...,/root/.cache/huggingface/datasets/downloads/ex...,"{'bytes': None, 'path': '/root/.cache/huggingf...",The river is crossed by the Murray Valley High...,2,0,twenties,male,england,en,
4,4875a5fc17c3936321a179429a287bc6971775f7f8b5d2...,/root/.cache/huggingface/datasets/downloads/ex...,"{'bytes': None, 'path': '/root/.cache/huggingf...",This car park is available for users of both M...,2,0,fifties,female,us,en,


In [ ]:
expected_accent = shuffled_df['accent'][1]
sentence = shuffled_df['sentence'][1]
expected_accent

'us'

In [ ]:
import torch
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor
import torchaudio

model = Wav2Vec2ForSequenceClassification.from_pretrained("./model-wav2vec2-accent-classification")
processor = Wav2Vec2Processor.from_pretrained("./model-wav2vec2-accent-classification")

# Mover el modelo a la GPU si está disponible, de lo contrario usar la CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Ruta al nuevo archivo de audio
audio_path = shuffled_df['path'][1]

# Preprocesar el nuevo audio
def preprocess_audio(audio_path):
    waveform, sample_rate = torchaudio.load(audio_path)

    if sample_rate != processor.feature_extractor.sampling_rate:
        resampler = torchaudio.transforms.Resample(sample_rate, processor.feature_extractor.sampling_rate)
        waveform = resampler(waveform)

    max_length = int(processor.feature_extractor.sampling_rate * 10)  # Ejemplo: 8 segundos
    inputs = processor(waveform.squeeze().numpy(),
                       sampling_rate=processor.feature_extractor.sampling_rate,
                       return_tensors="pt",
                       padding="max_length",
                       truncation=True,
                       max_length=max_length)

    return inputs

# Preprocesar el nuevo audio
inputs = preprocess_audio(audio_path)

# Mover los inputs al mismo dispositivo que el modelo
inputs = {k: v.to(device) for k, v in inputs.items()}

# Hacer predicciones
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=-1).item()

# Mapear la clase predicha al nombre del acento
id2label = {0: 'us', 1: 'england'}
predicted_accent = id2label[predicted_class_id]

print(f"The predicted accent is: {predicted_accent}")


The predicted accent is: england


In [ ]:
from IPython.display import Audio

# Ruta del archivo de audio en la primera fila del dataframe balanceado
audio_path = shuffled_df['path'][1]

# Reproducir el archivo de audio
Audio(audio_path)
